Before you turn this project in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Paul Tluczek"
COLLABORATORS = ""

# Load Data

The following code loads in the MNIST dataset and displays a few images and flattens the images and sets some autograder variables.

In [1]:
from IPython.display import display
import numpy as np
from keras.preprocessing.image import array_to_img, img_to_array
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
import tensorflow as tf
sess = tf.InteractiveSession()
%load_ext autoreload

[X_mnist, y_mnist], _ = mnist.load_data()
for x in X_mnist[:5]:
    x = np.expand_dims(x, axis=-1)
    img = array_to_img(x)
    display(img)

X_mnist = X_mnist.reshape([60_000, 28*28]) / 255.
Y_mnist = to_categorical(y_mnist)
X_mnist, Y_mnist = X_mnist[:50], Y_mnist[:50]

M, N = X_mnist.shape
C = np.unique(y_mnist).shape[0]
H = 16

def passed(): print('✅')

Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


# Task

- Define a one-hidden layer perceptron class called `TFMLP` with TensorFlow as in the computational graph

![](images/mlp_predict.svg)

In [2]:
%%writefile -a classifiers.py

import tensorflow as tf

class TFMLP():    
    def __init__(self, nb_feature, nb_hidden, nb_class):
        self.X = tf.placeholder(tf.float32, [None, nb_feature], name = 'X')
        self.Y = tf.placeholder(tf.float32, [None, nb_class], name = 'Y')
        self.W1 = tf.Variable(initial_value=np.random.randn(nb_feature, nb_hidden), dtype=tf.float32, name = 'W1')
        self.b1 = tf.Variable(initial_value=np.random.randn(nb_hidden), dtype=tf.float32, name = 'b1')        
        self.W2 = tf.Variable(initial_value=np.random.randn(nb_hidden, nb_class), dtype=tf.float32, name = 'W2')
        self.b2 = tf.Variable(initial_value=np.random.randn(nb_class), dtype=tf.float32, name = 'b2')
        
        self._f()
        
    def _f(self):
        Z = tf.add(tf.matmul(self.X, self.W1), self.b1)
        H = tf.div(tf.constant(1.0),
                    tf.add(tf.constant(1.0), tf.exp(tf.negative(Z))))
        S = tf.add(tf.matmul(H, self.W2), self.b2)  
        self.P = tf.nn.softmax(S)        
        correct_prediction = tf.equal(tf.argmax(self.Y,axis=1), tf.argmax(self.P,axis=1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
        self.loss = tf.reduce_mean(-tf.reduce_sum(self.Y * tf.log(self.P), reduction_indices=[1]))        
       
    def predict(self, X):       
        return self.P.eval(feed_dict={self.X:X})

    def evaluate(self, X, Y):  
        self.P.eval(feed_dict={self.X:X})
        return self.accuracy.eval(feed_dict={self.X:X, self.Y:Y})
    
    def forward(self, X, Y):
        return self.loss.eval(feed_dict={self.X:X, self.Y:Y})
        

Appending to classifiers.py


# Input Tests

In [3]:
%autoreload
from classifiers import TFMLP

mlp = TFMLP(nb_feature=N, nb_hidden=H, nb_class=C)
init = tf.global_variables_initializer()
sess.run(init) # initialize tensorflow variables

assert mlp.X.op.type == 'Placeholder'
assert mlp.Y.op.type == 'Placeholder'
mlp.X.get_shape().as_list() == [None, 784]
mlp.Y.get_shape().as_list() == [None, 10]

passed()

✅


# Variable Tests

In [4]:
from classifiers import TFMLP

mlp = TFMLP(nb_feature=N, nb_hidden=H, nb_class=C)
init = tf.global_variables_initializer()
sess.run(init) # initialize tensorflow variables

assert mlp.W1.op.type == 'VariableV2'
assert mlp.b1.op.type == 'VariableV2'
assert mlp.W1.get_shape().as_list() == [784, 16]
assert mlp.b1.get_shape().as_list() == [16]
assert mlp.W2.op.type == 'VariableV2'
assert mlp.b2.op.type == 'VariableV2'
assert mlp.W2.get_shape().as_list() == [16, 10]
assert mlp.b2.get_shape().as_list() == [10]

passed()

✅


# Output Tests

In [5]:
from classifiers import TFMLP

mlp = TFMLP(nb_feature=N, nb_hidden=H, nb_class=C)
init = tf.global_variables_initializer()
sess.run(init) # initialize tensorflow variables

assert mlp.loss.get_shape().as_list() == []
assert mlp.P.get_shape().as_list() == [None, 10]
assert mlp.accuracy.get_shape().as_list() == []

passed()

✅


# Prediction Tests

In [6]:
from classifiers import TFMLP

mlp = TFMLP(nb_feature=N, nb_hidden=H, nb_class=C)
init = tf.global_variables_initializer()
sess.run(init) # initialize tensorflow variables

S = mlp.predict(X_mnist)
nb_train_ = len(X_mnist)
assert S.shape == (M, C)

passed()

✅


# Evaluation Test

In [7]:
from classifiers import TFMLP

mlp = TFMLP(nb_feature=N, nb_hidden=H, nb_class=C)
init = tf.global_variables_initializer()
sess.run(init) # initialize tensorflow variables

acc = mlp.evaluate(X_mnist, Y_mnist)
assert type(acc) == tf.float32
assert 0 <= acc <= 1

passed()

✅


# Task

- Implement a `TFMLPWithGDOptimizer` class which performs optimization via gradient descent and extends your `TFMLP` class

In [8]:
%%writefile -a classifiers.py

class TFMLPWithGDOptimizer(TFMLP):    
    def __init__(self, nb_feature, nb_hidden, nb_class):
        TFMLP.__init__(self, nb_feature, nb_hidden, nb_class)
       
    def fit(self, X, Y, sess, nb_epoch=10, learningrate=0.5):
        train_step = tf.train.GradientDescentOptimizer(learning_rate=learningrate).minimize(self.loss)
        for _ in range(nb_epoch):          
            sess.run(fetches=[train_step, self.loss, self.accuracy], feed_dict={self.X: X, self.Y: Y})    


Appending to classifiers.py


# Gradient Descent Optimizer Test

In [9]:
%autoreload
from classifiers import TFMLPWithGDOptimizer, TFMLP

assert issubclass(TFMLPWithGDOptimizer, TFMLP)
mlp = TFMLPWithGDOptimizer(nb_feature=N, nb_hidden=H, nb_class=C)
init = tf.global_variables_initializer()
sess.run(init) # initialize tensorflow variables

X_sample, Y_sample = X_mnist[:50], Y_mnist[:50]
acc = mlp.evaluate(X_sample, Y_sample)
loss = mlp.forward(X_sample, Y_sample)
for _ in range(10):
    mlp.fit(X_sample, Y_sample, sess, nb_epoch=10)
    assert mlp.forward(X_sample, Y_sample) < loss
    loss = mlp.forward(X_sample, Y_sample)
    
assert mlp.evaluate(X_sample, Y_sample) > acc

passed()

✅


# Ignore Cell Below